In [1]:
import numpy as np
import sys, os, random, itertools, warnings, math
import matplotlib.pyplot as plt
from ase import Atoms, Atom
from ase.build import fcc100, fcc111, fcc110, bcc100, bcc111, bcc110, add_adsorbate, rotate
from ase.calculators.emt import EMT
from ase.calculators.vasp import Vasp, Vasp2
from ase.calculators.singlepoint import SinglePointCalculator as SPC
from ase.constraints import FixAtoms
from ase.eos import EquationOfState
from ase.geometry import find_mic
from ase.io import read, write
from ase.io.trajectory import Trajectory, TrajectoryWriter
from ase.lattice.cubic import FaceCenteredCubic
from ase.optimize import QuasiNewton
from ase.visualize import view
from scipy.spatial.qhull import QhullError
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.adsorption import AdsorbateSiteFinder

In [22]:
import numpy as np
import sys
import ktms

env='local'
# env = 'spacom'

a = 2.91
b = a * 2**0.5

def getatoms(a):
    b = a * 2**0.5
    x = a/2
    y = b/2
    z = b/2

    pos = ktms.np.array([[0,y,0],[0,0,z],[x,0,0],[x,y,z]])

    atoms = ktms.Atoms(symbols = 'Cu2Ni2', # Ga2Ni2
                       positions= pos,
                       cell = ktms.np.array([[x*2,0,0],
                                             [0,y*2,0],
                                             [0,0,z*2]]),
                       pbc=1
                       )
    return atoms


# latticeconstant = getLC(atoms, vaspset, env)
volumes = []
energies = []
cells = []

testrange = np.linspace(0.95, 1.05, 9)

for i in a*testrange:
    atoms = getatoms(i)

    if env == 'local':
        atoms.set_calculator(EMT())
        dyn = QuasiNewton(atoms)
        dyn.run(fmax=0.05)
    elif env == 'spacom':
        kpoints = ktms.getkpts(atoms)
        nb = ktms.getnbands(atoms, 2) # default value is 0.5
        vaspset = ktms.Vasp(
                    xc = 'PBE',
                    gga = 'RP',
                    ncore = 4,
                    encut = 500,
                    nsw = 200,
                    kpts = kpoints,
                    ibrion = 2,
                    isif = 2,
                    ismear = 1,
                    sigma = 0.2,
                    nbands = nb,
                    #algo = 'Normal', # 'All' 'Damped'
                    prec = 'Normal', # 'Accurate'
                    lcharg = False,
                    lwave = False,
                    )
        atoms.set_calculator(vaspset)

    try:
        ene = atoms.get_potential_energy()
        energies.append(ene)
        volumes.append(atoms.get_volume())
        cells.append(atoms.get_cell())

    except:
        print('Error while calculating bulk energy!')

eos = EquationOfState(volumes, energies)
v0, e0, B = eos.fit()
latticeconstant = (v0/2.0)**(1.0/3.0)

print(latticeconstant)

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 08:50:29        0.312186*       0.0000
                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 08:50:29        0.248128*       0.0000
                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 08:50:30        0.251133*       0.0000
                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 08:50:30        0.313375*       0.0000
                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 08:50:30        0.427665*       0.0000
                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optim